In [1]:
import numpy as np
import mujoco
import mediapy as media
import copy
import scipy
from robot_descriptions.skydio_x2_mj_description import MJCF_PATH

In [2]:
print(MJCF_PATH)

/home/simeon/.cache/robot_descriptions/mujoco_menagerie/skydio_x2/x2.xml


In [2]:
model = mujoco.MjModel.from_xml_path(MJCF_PATH)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model, height=480, width=640)

for act_id in range(4):
    model.actuator(act_id).ctrlrange = np.array([-1e4, 1e4])

model.opt.timestep = 1e-3

In [3]:
def compute_gains(configuration):
    # 1. Set configuration and find control that stabilizes it (ctrl0)
    newdata = mujoco.MjData(model)
    newdata = copy.copy(data)

    mujoco.mj_resetData(model, newdata)
    newdata.qpos = configuration
    # compute the control
    mujoco.mj_forward(model, newdata)
    newdata.qacc = 0
    mujoco.mj_inverse(model, newdata)

    # define control and configuration to linearize around
    ctrl0 = newdata.qfrc_inverse.copy() @ np.linalg.pinv(newdata.actuator_moment)
    qpos0 = newdata.qpos.copy()
    print(ctrl0, qpos0)

    # 2. Set weights
    R = np.eye(4) * 1e-3
    Q = np.block(
        [
            [np.eye(6), np.zeros((6, 6))],
            [np.zeros((6, 6)), np.eye(6)],
        ]
    )
    Q = np.eye(12)

    # 3. Compute LQR gains given weights
    mujoco.mj_resetData(model, newdata)
    newdata.ctrl = ctrl0
    newdata.qpos = qpos0

    # 4. Allocate the A and B matrices, compute them.
    A = np.zeros((2 * model.nv, 2 * model.nv))
    B = np.zeros((2 * model.nv, model.nu))
    epsilon = 1e-6
    flg_centered = True
    mujoco.mjd_transitionFD(model, newdata, epsilon, flg_centered, A, B, None, None)

    # Solve discrete Riccati equation.
    P = scipy.linalg.solve_discrete_are(A, B, Q, R)

    # Compute the feedback gain matrix K.
    K = np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A

    return ctrl0, K

In [ ]:
def Y_body(v_lin, v_ang, a_lin, a_ang):
    # this is identical to the pinnochio: pin.bodyRegressor(*[v_lin, v_ang], *[a_lin, a_ang])
    v1, v2, v3 = v_lin
    v4, v5, v6 = v_ang

    a1, a2, a3 = a_lin
    a4, a5, a6 = a_ang

    # fmt: off
    Y = np.array([
        [a1 - v2*v6 + v3*v5, -v5**2 - v6**2, -a6 + v4*v5, a5 + v4*v6, 0, 0, 0, 0, 0, 0],
        [a2 + v1*v6 - v3*v4, a6 + v4*v5, -v4**2 - v6**2, -a4 + v5*v6, 0, 0, 0, 0, 0, 0],
        [a3 - v1*v5 + v2*v4, -a5 + v4*v6, a4 + v5*v6, -v4**2 - v5**2, 0, 0, 0, 0, 0, 0],
        [0, 0, a3 - v1*v5 + v2*v4, -a2 - v1*v6 + v3*v4, a4, a5 - v4*v6, -v5*v6, a6 + v4*v5, v5**2 - v6**2, v5*v6],
        [0, -a3 + v1*v5 - v2*v4, 0, a1 - v2*v6 + v3*v5, v4*v6, a4 + v5*v6, a5, -v4**2 + v6**2, a6 - v4*v5, -v4*v6],
        [0, a2 + v1*v6 - v3*v4, -a1 + v2*v6 - v3*v5, 0, -v4*v5, v4**2 - v5**2, v4*v5, a4 - v5*v6, a5 + v4*v6, a6]
    ])
    # fmt: on

    return Y


def mj_bodyRegressor(mj_model, mj_data, body_id):
    velocity = np.zeros(6)
    accel = np.zeros(6)
    _cross = np.zeros(3)

    mujoco.mj_objectVelocity(mj_model, mj_data, 2, body_id, velocity, 1)
    mujoco.mj_rnePostConstraint(mj_model, mj_data)
    mujoco.mj_objectAcceleration(mj_model, mj_data, 2, body_id, accel, 1)
    # rotation = mj_data.xmat[body_id].reshape(3, 3).copy()

    v, w = velocity[3:], velocity[:3]
    dv, dw = accel[3:], accel[:3]  # dv - classical acceleration, already containt g
    mujoco.mju_cross(_cross, w, v)
    
    # in floating body, should be canceled?
    # dv -= _cross
    # print(v, w)
    # print(dv, dw)
    
    Y = Y_body(v, w, dv, dw)
    return Y


In [4]:
# Parameters.
DURATION = 20  # seconds
FRAMERATE = 20  # Hz
BALANCE_STD = 0.3  # actuator units

dq = np.zeros(model.nv)

qpos0 = np.array([0, 0, 0.3, 1.0, 0, 0, 0])
zero_config = qpos0.copy()
ctrl0, K = compute_gains(zero_config)


# Reset data, set initial pose.
mujoco.mj_resetData(model, data)
data.qpos = qpos0

last_time = 0
phase = 0


frames = []
step = 0
while data.time < DURATION:
    # Get state difference dx.
    mujoco.mj_differentiatePos(model, dq, 1, zero_config, data.qpos)
    dx = np.hstack((dq, data.qvel)).T

    # LQR control law.
    data.ctrl = ctrl0 - K @ dx
    # data.xfrc_applied[1, :3] = np.random.randn(3) * BALANCE_STD

    # Step the simulation.
    mujoco.mj_step(model, data)

    if data.time > phase * 2.0:
        phase += 1
        zero_config[phase % 3] = 0.3
        zero_config[(phase + 1) % 3] = 0
        zero_config[(phase + 2) % 3] = 0
        ctrl0, K = compute_gains(zero_config)

    # Render and save frames.
    if len(frames) < data.time * FRAMERATE:
        renderer.update_scene(data)
        pixels = renderer.render()
        frames.append(pixels)

media.show_video(frames, fps=FRAMERATE)

[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.  0.3 1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.3 0.  1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.  0.3 1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.3 0.  0.  1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.3 0.  1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.  0.3 1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.3 0.  0.  1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.3 0.  1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.  0.3 1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.3 0.  0.  1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.3 0.  1.  0.  0.  0. ]
[3.2495625 3.2495625 3.2495625 3.2495625] [0.  0.  0.3 1.  0.  0.  0. ]


This browser does not support the video tag.


In [5]:
data.qpos

array([0.0504273 , 0.23954211, 0.00493455, 0.99982789, 0.00384175,
       0.0042072 , 0.01765585])